In [86]:
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [135]:
os.chdir('/Users/wyf/Downloads/titanic')
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')


In [136]:
train = train.fillna(29.7) # mean age and the % of survived among age-null is close to age band 16-32
test = test.fillna(29.7)
full = [train, test] #just like a set with two dataset, full[0] is train, full[1] is test

In [137]:
for dataset in full:
    dataset['title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand = False)
for dataset in full:
    dataset['ticket_sign'] = dataset.Ticket.str.extract('([0-9]+)', expand = False) 
for dataset in full:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0  # for the row whose age <=16 in age column
    dataset.loc[ (dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[ (dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2        
    dataset.loc[ (dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
for dataset in full:
    dataset['Pclass*Age'] = dataset.Age * dataset.Pclass
for dataset in full:
    dataset['loneliness'] = 0
    dataset.loc[(dataset['SibSp'] == 0) & (dataset['Parch'] == 0) ,'loneliness'] = 1

In [138]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title,ticket_sign,Pclass*Age,loneliness
0,1,0,3,"Braund, Mr. Owen Harris",male,1.0,1,0,A/5 21171,7.2500,29.7,S,Mr,5,3.0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2.0,1,0,PC 17599,71.2833,C85,C,Mrs,17599,2.0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,1.0,0,0,STON/O2. 3101282,7.9250,29.7,S,Miss,2,3.0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,2.0,1,0,113803,53.1000,C123,S,Mrs,113803,2.0,0
4,5,0,3,"Allen, Mr. William Henry",male,2.0,0,0,373450,8.0500,29.7,S,Mr,373450,6.0,1


In [151]:
train_ready = train.drop(['Cabin','PassengerId','Name','Ticket', 'ticket_sign','SibSp','Parch'], axis=1)
train_ready['Sex'] = pd.factorize(train_ready.Sex)[0]
train_ready['Embarked'] = pd.factorize(train_ready.Embarked)[0]
train_ready['title'] = pd.factorize(train_ready.title)[0]
test_ready = test.drop(['Cabin','PassengerId','Name','Ticket', 'ticket_sign','SibSp','Parch'], axis=1)
test_ready['Sex'] = pd.factorize(test_ready.Sex)[0]
test_ready['Embarked'] = pd.factorize(test_ready.Embarked)[0]
test_ready['title'] = pd.factorize(test_ready.title)[0]

In [143]:
train_ready.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,title,Pclass*Age,loneliness
0,0,3,0,1.0,7.2500,0,0,3.0,0
1,1,1,1,2.0,71.2833,1,1,2.0,0
2,1,3,1,1.0,7.9250,0,2,3.0,1
3,1,1,1,2.0,53.1000,0,1,2.0,0
4,0,3,0,2.0,8.0500,0,0,6.0,1


In [152]:
test_ready.head()

,Pclass,Sex,Age,Fare,Embarked,title,Pclass*Age,loneliness
0,3,0,2.0,7.8292,0,0,6.0,1
1,3,1,2.0,7.0000,1,1,6.0,0
2,2,0,3.0,9.6875,0,0,6.0,1
3,3,0,1.0,8.6625,1,0,3.0,1
4,3,1,1.0,12.2875,1,1,3.0,0


In [144]:
x = train_ready.iloc[:,1:].values
y = train_ready["Survived"].values

In [153]:
SEED = 42
cross_valid_scores = {}

parameters = {
    "n_estimators": [5, 10, 15, 20, 25, 100], 
    "max_depth": [3, 5, 7, 9, 11, 13],
}
model_random_forest = RandomForestClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_random_forest = GridSearchCV(
    model_random_forest, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_random_forest.fit(x, y)

print('-----')
print(f'Best parameters {model_random_forest.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_random_forest.best_score_:.3f}'
)
cross_valid_scores['random_forest'] = model_random_forest.best_score_
print('-----')

-----
Best parameters {'max_depth': 7, 'n_estimators': 25}
Mean cross-validated accuracy score of the best_estimator: 0.835
-----


In [150]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.30,random_state=8)

logreg = LogisticRegression()
logreg.fit(xtrain, ytrain)
ypred = logreg.predict(xtest)
logreg.score(xtrain, ytrain)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7961476725521669

In [149]:
parameters = {"max_depth": [3,5,7,9,11,13]}

model_dt = DecisionTreeClassifier(
    random_state = SEED,
    class_weight = 'balanced',
)

model_dt = GridSearchCV(
    model_dt,
    parameters,
    cv = 5,
    scoring = 'accuracy',
)

model_dt.fit(x, y)

print('-------')
print(f'Best parameters {model_dt.best_params_}')
print(f'Mean accuracy:'+\
     f'{model_dt.best_score_:.4f}')

cross_valid_scores['decision_tree'] = model_dt.best_score_
print('-------')

-------
Best parameters {'max_depth': 5}
Mean accuracy:0.8227
-------


In [155]:
result = model_random_forest.predict(test_ready)

In [156]:
len(result)

418

In [158]:
clf1 = RandomForestClassifier(n_estimators=25, max_depth=7)
clf1 = clf1.fit(x,y)
prediction1 = clf1.predict(test_ready)
result == prediction1

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [159]:
prediction1

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [163]:
submission = pd.DataFrame({
    "PassengerId": test['PassengerId'],
    "Survived": result
})
submission.to_csv('submission.csv', index = False)